In [5]:
import pandas as pd
import numpy as np
import torch 

In [20]:
hartwig = pd.read_csv('hartwig.csv')

In [19]:
hartwig.columns

Index(['Game Number', 'Move Number', 'Player', 'Move', 'Comment'], dtype='object')

In [23]:
hartwig

,Game Number,Move Number,Player,Move,Comment
0,1,1,White,e4 e5,NaN
1,1,2,Black,f4 Bc5,NaN
2,1,3,White,Nf3 d6,NaN
3,1,4,Black,Nc3 Nc6,NaN
4,1,5,White,Bb5 Bg4,"The first four moves might be out of fashion, ..."
...,...,...,...,...,...
561,15,37,White,f3 Nf5,NaN
562,15,38,Black,Rxf5 Rxf5,NaN
563,15,39,White,Rd7+ Kg6,NaN
564,15,40,Black,Rxb7 Rf4,NaN


In [22]:
import torch
from transformers import BertTokenizer, GPT2LMHeadModel

# Define model and tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")
model = GPT2LMHeadModel.from_pretrained("gpt2")

# Hyperparameters
learning_rate = 2e-5
epochs = 5

# Training loop
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
for epoch in range(epochs):
  for board, move, commentary in train_data:
    # Preprocess board, move, and commentary
    encoded_board = tokenizer(board, return_tensors="pt")
    encoded_move = torch.tensor([move]).unsqueeze(0)

    # Encode board and move
    with torch.no_grad():
      encoder_outputs = model.bert.encoder(encoded_board)

    # Get last hidden state from encoder (encoded representation)
    encoded_representation = encoder_outputs.last_hidden_state

    # Decoder input (start token)
    decoder_input = torch.tensor([tokenizer.encode("<s>")], dtype=torch.long)

    # Generate commentary one word at a time
    for comment_word in commentary:
      # Get decoder outputs and logits
      decoder_outputs = model.gpt2(decoder_input, encoder_outputs=encoded_representation)
      logits = decoder_outputs.logits[:, -1, :]

      # Sample next word based on logits
      next_word = torch.argmax(logits, dim=-1).unsqueeze(0)
      decoder_input = torch.cat((decoder_input, next_word), dim=-1)

    # Calculate loss
    labels = torch.tensor(tokenizer.encode(commentary), dtype=torch.long)[1:]  # Ignore start token
    loss = torch.nn.functional.cross_entropy(logits.view(-1, model.vocab_size), labels)

    # Backpropagate and update weights
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Print training progress (optional)
    print(f"Epoch: {epoch+1}, Loss: {loss.item()}")


566